This is a demo of how we can add catalog simulations to LSST alerts in avro forma, which the Rubin project is using. 

Follow the README of this repository and then the README of `alert_packet` for setup. This involves installing `alert_packet` in some way.

In [ ]:
This notebook is based on the examples and codes in `alert_packet` :https://github.com/lsst/alert_packet

In [1]:
import lsst.alert.packet

In [2]:
from pathlib import Path

In [3]:
import json

In [4]:
schema = lsst.alert.packet.Schema.from_file()

If you clone the repository `alert_packet` this is the location of the alert.json file.

In [5]:
path = Path('../../alert_packet/python/lsst/alert/packet/schema/4/0/sample_data/')
with open(path/'alert.json') as f:
    alert_data = json.load(f)

In [6]:
alert_data

{'alertId': 1231321321,
 'l1dbId': 222222222,
 'diaSource': {'diaSourceId': 281323062375219200,
  'ccdVisitId': 111111,
  'midPointTai': 1480360995,
  'filterName': 'r',
  'programId': 1,
  'ra': 351.570546978,
  'decl': 0.126243049656,
  'raSigma': 0.00028,
  'declSigma': 0.00028,
  'ra_decl_Cov': 0.00029,
  'x': 112.1,
  'y': 121.1,
  'xSigma': 1.2,
  'ySigma': 1.1,
  'x_y_Cov': 1.2,
  'apFlux': 1241.0,
  'apFluxErr': 12.0,
  'snr': 41.1,
  'psFlux': 1241.0,
  'psFluxErr': 12.0,
  'flags': 0},
 'prvDiaSources': [{'diaSourceId': 281323062375219198,
   'ccdVisitId': 111111,
   'midPointTai': 1480360995,
   'filterName': 'r',
   'programId': 1,
   'ra': 351.570546978,
   'decl': 0.126243049656,
   'raSigma': 0.00028,
   'declSigma': 0.00028,
   'ra_decl_Cov': 0.00029,
   'x': 112.1,
   'y': 121.1,
   'xSigma': 1.2,
   'ySigma': 1.1,
   'x_y_Cov': 1.2,
   'apFlux': 1241.0,
   'apFluxErr': 12.0,
   'snr': 41.1,
   'psFlux': 1241.0,
   'psFluxErr': 12.0,
   'flags': 0},
  {'diaSourceId': 2

## What does the data corresponding to a diasrc look like? 

This is just a dictionary: You can write some code to read the data from SNANA and write it to the values of the correct keys

In [7]:
diasrc = alert_data['prvDiaSources'][0]

In [8]:
from copy import copy

Example:

In [9]:
my_diasrc = copy(diasrc)

In [10]:
my_diasrc['psFlux'] = 1531.2
my_diasrc['snr'] = my_diasrc['psFlux']/my_diasrc['psFluxErr']# Not sure this is correct !

In [11]:
print(diasrc)

{'diaSourceId': 281323062375219198, 'ccdVisitId': 111111, 'midPointTai': 1480360995, 'filterName': 'r', 'programId': 1, 'ra': 351.570546978, 'decl': 0.126243049656, 'raSigma': 0.00028, 'declSigma': 0.00028, 'ra_decl_Cov': 0.00029, 'x': 112.1, 'y': 121.1, 'xSigma': 1.2, 'ySigma': 1.1, 'x_y_Cov': 1.2, 'apFlux': 1241.0, 'apFluxErr': 12.0, 'snr': 41.1, 'psFlux': 1241.0, 'psFluxErr': 12.0, 'flags': 0}


In [12]:
print(my_diasrc)

{'diaSourceId': 281323062375219198, 'ccdVisitId': 111111, 'midPointTai': 1480360995, 'filterName': 'r', 'programId': 1, 'ra': 351.570546978, 'decl': 0.126243049656, 'raSigma': 0.00028, 'declSigma': 0.00028, 'ra_decl_Cov': 0.00029, 'x': 112.1, 'y': 121.1, 'xSigma': 1.2, 'ySigma': 1.1, 'x_y_Cov': 1.2, 'apFlux': 1241.0, 'apFluxErr': 12.0, 'snr': 127.60000000000001, 'psFlux': 1531.2, 'psFluxErr': 12.0, 'flags': 0}


## Converting to efficient avro format

Now with the entire `alert_data` file, we have

In [13]:
avro_bytes = schema.serialize(alert_data)

In [14]:
messg = schema.deserialize(avro_bytes)